<a href="https://colab.research.google.com/github/z00m-in/deeplearning_zero_to_all_2/blob/main/%EB%AA%A8%EB%91%90%EB%94%A52_Lab07_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np

기본 데이터 세팅

In [ ]:
xy = np.array([[828.659973, 833.450012, 908100, 828.349976, 831.659973],
               [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
               [819.929993, 824.400024, 1438100, 818.97998, 824.159973],
               [816, 820.958984, 1008100, 815.48999, 819.23999],
               [819.359985, 823, 1188100, 818.469971, 818.97998],
               [819, 823, 1198100, 816, 820.450012],
               [811.700012, 815.25, 1098100, 809.780029, 813.669983],
               [809.51001, 816.659973, 1398100, 804.539978, 809.559998]])

x_train = xy[:, 0:-1]
y_train = xy[:, [-1]]

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))
W = tf.Variable(tf.random_normal([4, 1]), dtype=tf.float32)
b = tf.Variable(tf.random_normal([1]), dtype=tf.float32)

정규화: Normalization

In [ ]:
# 정규화 실행
def normalization(data):
  numerator = data - np.min(data, 0)
  denominator = np.max(data, 0) - np.min(data, 0)
  return numerator / denominator

xy = normalization(xy)

L2 Normalization

In [ ]:
def linearReg_fn(features):
  hypothesis = tf.matmul(features, W) + b
  return hypothesis

In [ ]:
# 기존의 cost function에 λ라는 특정 값을 추가해 regularization이 실현되는 경우.
# 함수의 beta가 λ에 해당
def l2_loss(loss, beta = 0.01):
  W_reg = tf.nn.l2_loss(W) # output = sum(t ** 2) / 2
  loss = tf.reduce_mean(loss + W_reg * beta)
  return loss

In [ ]:
def loss_fn(hypothesis, labels, flag = False):
  cost = tf.reduce_mean(tf.square(hypothesis - labels))
  if(flag):
    cost = l2_loss(cost)
  return cost

Learning Decay

In [ ]:
is_decay = True
starter_learning_rate = 0.1

if(is_decay):
  global_step = tf.Variable(0, trainable=False)
  # exponential_decay(시작값, 전체 step, 몇 번의 step 마다, 얼만큼의 값을 조절할지)
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 50, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
else:
  optimizer = tf.train.GradientDescentOptimizer(learning_rate=starter_learning_rate)

In [ ]:
def grad(features, labels, l2_flag):
 with tf.GradientTape() as tape:
   # 가설의 결과와 실제 값을 비교하면서 loss 값 구하고 L2 flag 조정
   loss_value = loss_fn(linearReg_fn(features),labels, l2_flag)
 return tape.gradient(loss_value, [W,b]), loss_value

In [ ]:
# 실제 실행 부분
for step in range(EPOCHS):
  for features, labels in tfe.Iterator(dataset):
    features = tf.cast(features, tf.float32)
    labels = tf.cast(labels, tf.float32)
    grads, loss_value = grad(linearReg_fn(features), features, labels, False)
    optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]), global_step=global_step)
    if step % 10 == 0:
      print("Iter: {}, Loss: {:.4f}, Learning Rate: {:.8f}".format(step, loss_value, optimizer._learning_rate()))